In [37]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import random
pio.templates.default = 'simple_white'
import datetime
import tqdm

In [38]:
raw_df = pd.read_csv('weekly_data.csv')
raw_df.head()

,user_id,platform,week_date,cohort_week
0,1000001,web,2020-12-28,2020-12-28
1,1000001,web,2021-01-04,2020-12-28
2,1000001,web,2021-01-11,2020-12-28
3,1000001,web,2021-02-15,2020-12-28
4,1000001,web,2021-02-22,2020-12-28


In [47]:
df = raw_df.groupby(['week_date', 'cohort_week'], as_index = False).user_id.nunique()
df = df[df.week_date <= '2023-07-01']

In [48]:
df.shape[0]

8635

In [49]:
df['week_num'] = list(map(
    lambda d, c: (datetime.datetime.strptime(d, '%Y-%m-%d') - datetime.datetime.strptime(c, '%Y-%m-%d')).days,
    df.week_date,
    df.cohort_week
)) 

In [51]:
px.line(
    df[df['week_num'] == 0].sort_values('cohort_week').set_index('cohort_week')[['user_id']],
    title = 'New users'
)

In [57]:
px.line(df.groupby('week_date').user_id.sum(), 
       title = 'Active users')

In [64]:
retention_df = df[df.cohort_week >='2022-07-01'].pivot(index='week_num', columns = 'cohort_week',
                                       values = 'user_id')

px.line(retention_df.apply(lambda x: 100*x/retention_df.iloc[0], axis = 1))